<div class="page--post">
  <header class="post__header">
  <div class="heading heading--alignment-left">
  <p class="heading__overhead alternative--caption">Deep Learning</p>
    <h1>How ChatGPT actually works</h1>
  <p class="heading__subtitle body--large">Since its release, the public has been playing with ChatGPT and seeing what it can do, but how does ChatGPT actually work? While the details of its inner workings have not been published, we can piece together its functioning principles from recent research.</p>
</div> 
</header>
  <article class="post__body">
    <section class="post__content">
      <p>ChatGPT is the latest language model from OpenAI and represents a significant improvement over its predecessor GPT-3. Similarly to many Large Language Models, ChatGPT is capable of generating text in a wide range of styles and for different purposes, but with remarkably greater precision, detail, and coherence. It represents the next generation in OpenAI's line of Large Language Models, and it is designed with a strong focus on interactive conversations. </p><p>The creators have used a combination of both Supervised Learning and Reinforcement Learning to fine-tune ChatGPT, <strong>but it is the Reinforcement Learning component specifically that makes ChatGPT unique</strong>. The creators use a particular technique called Reinforcement Learning from Human Feedback (RLHF), which uses human feedback in the training loop to minimize harmful, untruthful, and/or biased outputs.</p><p>We are going to examine GPT-3's limitations and how they stem from its training process, before learning how RLHF works and <strong>understand how ChatGPT uses RLHF to overcome these issues</strong>. We will conclude by looking at some of the limitations of this methodology.</p><h2 id="capability-vs-alignment-in-large-language-models"><a href="#capability-vs-alignment-in-large-language-models" class="anchor"></a>Capability vs Alignment in Large Language Models</h2><figure class="kg-card kg-image-card kg-card-hascaption"><img src="https://www.assemblyai.com/blog/content/images/2022/12/capability-versus-alignment-in-machine-learning---how-chatGPT-works--1-.png" class="kg-image" alt="" loading="lazy" width="960" height="540" srcset="https://www.assemblyai.com/blog/content/images/size/w600/2022/12/capability-versus-alignment-in-machine-learning---how-chatGPT-works--1-.png 600w, https://www.assemblyai.com/blog/content/images/2022/12/capability-versus-alignment-in-machine-learning---how-chatGPT-works--1-.png 960w" sizes="(min-width: 720px) 720px"><figcaption>"alignment vs capability" can be thought of as a more abstract analogue of "accuracy vs precision"</figcaption></figure><p>In the context of machine learning, the term <em>capability</em> refers to a model's ability to perform a specific task or set of tasks. A model's capability is typically evaluated by <strong>how well it is able to optimize its objective function</strong>, the mathematical expression that defines the goal of the model. For example, a model designed to predict stock market prices might have an objective function that measures the accuracy of the model's predictions. If the model is able to accurately predict the movement of stock prices over time, it would be considered to have a high level of capability for this task.</p><p><em>Alignment,</em> on the other hand, is concerned with <strong>what we actually want the model to do</strong> versus what it is being trained to do. It asks the question “is that objective function consistent with our intentions?” and refers to the extent to which a model's goals and behavior align with human values and expectations. For a simple concrete example, say we train a bird classifier to classify birds as either "sparrows" or "robins" and we use <em>log loss</em> (which measures the difference between the predicted probability distribution of the model and the true distribution) as the training objective, even though our ultimate goal is a high classification accuracy. The model might have low log loss, i.e. <strong>the model’s capability is high, but poor accuracy</strong> on the test set. In fact, <a href="https://datascience.stackexchange.com/questions/42599/what-is-the-relationship-between-the-accuracy-and-the-loss-in-deep-learning/42611?ref=assemblyai.com">the log loss is not perfectly correlated with accuracy in classification tasks</a>. This is an example of misalignment, where the model is capable of optimizing the training objective but poorly aligned with our ultimate goal.</p><blockquote class="kg-blockquote-alt">Models like the original GPT-3 are misaligned</blockquote><p>Large Language Models, such as GPT-3, are trained on vast amounts of text data from the internet <strong>and are capable of generating human-like text</strong>, but they may not always produce output that is consistent with human expectations or desirable values. In fact, their objective function is a probability distribution over word sequences (or token sequences) that allows them to predict what the next word is in a sequence (more details on this below).</p><p>In practical applications, however, <strong>these models are intended to perform some form of valuable cognitive work</strong>, and there is a clear divergence between the way these models are trained and the way we would like to use them. Even though a machine calculated statistical distribution of word sequences might be, mathematically speaking, a very effective choice to model language, we as humans generate language by choosing text sequences that are best for the given situation, using our background knowledge and common sense to guide this process. This can be a problem when language models are used in applications that require a high degree of trust or reliability, such as dialogue systems or intelligent personal assistants. </p><p>While these powerful, complex models trained on huge amounts of data have become extremely capable in the last few years, when used in production systems to make human lives easier they often fall short of this potential. The alignment problem in Large Language Models typically manifests as:</p><ul><li>Lack of helpfulness: not following the user's explicit instructions.</li><li>Hallucinations: model making up unexisting or wrong facts.</li><li>Lack of interpretability: it is difficult for humans to understand how the model arrived at a particular decision or prediction.</li><li>Generating biased or toxic output: a language model that is trained on biased/toxic data may reproduce that in its output, even if it was not explicitly instructed to do so. </li></ul><p>But where does this alignment problem stem from, concretely? Is it <strong>the very way language models are trained inherently prone to misalignment?</strong></p><h2 id="how-language-model-training-strategies-can-produce-misalignment"><a href="#how-language-model-training-strategies-can-produce-misalignment" class="anchor"></a>How language model training strategies can produce misalignment</h2><p><code>Next-token-prediction</code> and <code>masked-language-modeling</code> are <strong>the core techniques used for training language models</strong>, such as <a href="https://en.wikipedia.org/wiki/Transformer_(machine_learning_model)?ref=assemblyai.com">transformers</a>. In the first approach, the model is given a sequence of words (or “tokens”, i.e. parts of words) as input and is asked to predict the next word in the sequence. For example, if the model is given the input sentence </p><p><em>"The cat sat on the"</em></p><p>it might predict the next word as <em>"mat"</em>, <em>"chair"</em>, or <em>"floor"</em>, because of the high-probability of occurrence of these words given the previous context; the language model is in fact able to <strong>estimate the likelihood of each possible word </strong>(in its vocabulary) given the previous sequence.</p><p>The masked language modeling approach is a variant of next token prediction, in which some of the words in the input sentence are replaced with a special token, such as <code>[MASK]</code>. The model is then asked to predict the correct word that should be inserted in place of the mask. For example, if the model is given the sentence </p><p><em>"The <code>[MASK]</code> sat on the"</em> </p><p>as input, it might predict the next word as <em>"cat"</em>, <em>"dog"</em>, or <em>"rabbit"</em>.</p><p>One advantage of these objective functions is that it allows the model to <strong>learn the statistical structure of language</strong>, such as common word sequences and patterns of word usage. This generally helps the model generate more natural and fluent text, and it is an essential step in the pre-training phase of every language model.</p><p>However, these objective functions can also lead to problems, essentially because the model is not capable of <strong>distinguishing between an important error and an unimportant one</strong>. To make a very simple example, if the model is given the input sentence:</p><p><em>"The Roman Empire <code>[MASK]</code> with the reign of Augustus."</em> </p><p>it might predict <em>"began"</em> or <em>“ended”</em>, as both words score high likelihood of occurrence (indeed, <a href="https://en.wikipedia.org/wiki/Augustus_(title)?ref=assemblyai.com">both sentences are historically correct</a>), even though the second choice implies a very different meaning.</p><p>More generally, these training strategies can lead to a misalignment of the language model for some more complex tasks, because a model which is only trained to predict the next word (or a masked word) in a text sequence, <strong>may not necessarily be learning some higher-level representations of its meaning</strong>. As a result, the model struggles to generalize to tasks or contexts that require a deeper understanding of language.</p><p>Researchers and developers are working on various approaches to address the alignment problem in Large Language Models. ChatGPT is based on the original GPT-3 model, but has been further trained by using human feedback to guide the learning process with the specific goal of mitigating the model’s misalignment issues. The specific technique used, called Reinforcement Learning from Human Feedback, is based on <a href="#references">previous academic research</a>. ChatGPT represents the <strong>first case of use of this technique for a model put into production</strong>.</p><p>But how exactly do the creators of ChatGPT use human feedback to attack the alignment problem?</p><h2 id="reinforcement-learning-from-human-feedback"><a href="#reinforcement-learning-from-human-feedback" class="anchor"></a>Reinforcement Learning from Human Feedback</h2><p>The method overall consists of three distinct steps:</p><ol><li><strong>Supervised fine-tuning</strong> – A pre-trained language model is fine-tuned on a relatively small amount of demonstration data curated by labelers, to learn a supervised <a href="https://www.baeldung.com/cs/ml-policy-reinforcement-learning?ref=assemblyai.com">policy</a> (the <strong>SFT model</strong>) that generates outputs from a selected list of prompts. This represents the baseline model.</li><li><strong>“Mimic human preferences”</strong> – Labelers are asked to vote on a relatively large number of the SFT model outputs, this way creating a new dataset consisting of <em>comparison data</em>. A new model is trained on this dataset. This is referred to as the <strong>reward model (RM)</strong>.</li><li><strong>Proximal Policy Optimization (PPO)</strong> – The reward model is used to further fine-tune and improve the SFT model. The outcome of this step is the so-called <strong>policy model</strong>.</li></ol><p>Step 1 takes place only once, while steps 2 and 3 can be iterated continuously: more comparison data is collected on the current best policy model, which is used to train a new reward model and then a new policy.</p><p>Let’s now dive into the details of each step!</p><p>Note: The rest of this article is based on the content of the <a href="https://arxiv.org/pdf/2203.02155.pdf?ref=assemblyai.com">InstructGPT paper</a>. According to OpenAI, ChatGPT has been trained <em>“using the same methods as InstructGPT, but with slight differences in the data collection setup”</em> (<a href="https://openai.com/blog/chatgpt/?ref=assemblyai.com">source</a>). Unfortunately, exact quantitative reports have yet to be made publicly available for ChatGPT.</p><h3 id="step-1-the-supervised-fine-tuning-sft-model">Step 1: The Supervised Fine-Tuning (SFT) model</h3><p>The first step consists in collecting demonstration data in order to train a supervised policy model, referred to as the SFT model.</p><ul><li>Data collection: a list of prompts is selected and a group of human labelers are asked to <strong>write down the expected output response</strong>. For ChatGPT, two different sources of prompts have been used: some have been prepared directly from the labelers or developers, some have been sampled from OpenAI’s API requests (i.e. from their GPT-3 customers). As this whole process is slow and expensive, the result is a relatively small, high-quality curated dataset (of approximately 12-15k data points, presumably) that is to be used to fine-tune a pretrained language model.</li><li>Choice of model: instead of fine-tuning the original GPT-3 model, the developers of ChatGPT opted for a pretrained model in the so-called <a href="https://beta.openai.com/docs/model-index-for-researchers?ref=assemblyai.com">GPT-3.5 series</a>. Presumably the baseline model used is the latest one <code>text-davinci-003</code>, a GPT-3 model which was fine-tuned mostly on programming code. </li></ul><p>Quite interestingly, therefore, in order to create a general purpose chatbot like ChatGPT, the developers decided to <strong>fine-tune on top of a “code model” </strong>rather than a pure text model.</p><figure class="kg-card kg-image-card kg-card-hascaption"><img src="https://www.assemblyai.com/blog/content/images/2022/12/ChatGPT-1.jpg" class="kg-image" alt="" loading="lazy" width="400" height="664"><figcaption>Figure adapted from <a href="https://openai.com/blog/chatgpt/?ref=assemblyai.com">source</a></figcaption></figure><p>Due to the limited amount of data for this step, the SFT model obtained after this process is likely to output text which is still (probabilistically) not very user-attentive and generally suffers from misalignment, in the sense explained in the above sections. The problem here is that <strong>the supervised learning step suffers from high scalability costs</strong>.</p><p>To overcome this problem, instead of asking human labelers to create a much bigger curated dataset, a slow and costly process, the strategy is now to have the labelers rank different outputs of the SFT model to create a reward model –let’s explain this in more detail in the following section.</p><h3 id="step-2-the-reward-model-rm">Step 2: The reward model (RM)</h3><p>The goal is to learn an objective function (the reward model) <strong>directly from the data</strong>. The purpose of this function is to give a score to the SFT model outputs, proportional to how desirable these outputs are for humans. In practice, this will strongly reflect the specific preferences of the selected group of human labelers and the common guidelines which they agreed to follow. In the end, this process will extract from the data an automatic system that is supposed to <strong>mimic human preferences</strong>. </p><p>Here’s how it works:</p><ul><li>A list of prompts is selected and the SFT model generates multiple outputs (anywhere between 4 and 9) for each prompt.</li><li>Labelers rank the outputs from best to worst. The result is a new labeled dataset, where the rankings are the labels. The size of this dataset is approximately 10 times bigger than the curated dataset used for the SFT model.</li><li>This new data is used to train a reward model (RM). This model takes as input a few of the SFT model outputs and ranks them in order of preference.</li></ul><figure class="kg-card kg-image-card kg-card-hascaption"><img src="https://www.assemblyai.com/blog/content/images/2022/12/ChatGPT-2.jpeg" class="kg-image" alt="" loading="lazy" width="400" height="729"><figcaption>Figure adapted from <a href="https://openai.com/blog/chatgpt/?ref=assemblyai.com">source</a></figcaption></figure><p>As for labelers it is much easier to rank the outputs than to produce them from scratch, <strong>this process scales up much more efficiently</strong>. In practice, this dataset has been generated from a selection of 30-40k prompts, and a variable number of the generated outputs (for each prompt) is presented to the each labeler during the ranking phase.</p><h3 id="step-3-fine-tuning-the-sft-model-via-proximal-policy-optimization-ppo">Step 3: Fine-tuning the SFT model via Proximal Policy Optimization (PPO)</h3><p><a href="https://www.assemblyai.com/blog/reinforcement-learning-with-deep-q-learning-explained/">Reinforcement Learning</a> is now applied to fine-tune the SFT policy by letting it optimize the reward model. The specific algorithm used is called Proximal Policy Optimization (PPO) and the fine-tuned model is referred to as the PPO model.</p><p>What is PPO? Here are the main takeaways of this method: </p><ul><li>PPO is an algorithm that is used to train agents in reinforcement learning. It is called an "on-policy" algorithm because <strong>it learns from and updates the current policy directly</strong>, rather than learning from past experiences as in "off-policy" algorithms like DQN (Deep Q-Network). This means that PPO is continuously adapting the current policy based on the actions that the agent is taking and the rewards it is receiving.</li><li>PPO uses a <a href="https://en.wikipedia.org/wiki/Trust_region?ref=assemblyai.com">trust region optimization method</a> to train the policy, which means that <strong>it constrains the change in the policy to be within a certain distance of the previous policy</strong> in order to ensure stability. This is in contrast to other policy gradient methods which can sometimes make large updates to the policy that can destabilize learning.</li><li>PPO uses a <strong>value function to estimate the expected return of a given state or action</strong>. The value function is used to compute the advantage function, which represents the difference between the expected return and the current return. The advantage function is then used to update the policy by comparing the action taken by the current policy to the action that would have been taken by the previous policy. This allows PPO to make more informed updates to the policy based on the estimated value of the actions being taken.</li></ul><p>In this step, the PPO model is initialized from the SFT model, and <strong>the value function is initialized from the reward model</strong>. The environment is a <a href="https://en.wikipedia.org/wiki/Multi-armed_bandit?ref=assemblyai.com">bandit environment</a> which presents a random prompt and expects a response to the prompt. Given the prompt and response, it produces a reward (determined by the reward model) and the episode ends. A per-token <a href="https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence?ref=assemblyai.com">KL penalty</a> is added from the SFT model at each token to mitigate over optimization of the reward model. <br></p><figure class="kg-card kg-image-card kg-card-hascaption"><img src="https://www.assemblyai.com/blog/content/images/2022/12/ChatGPT-3.jpeg" class="kg-image" alt="" loading="lazy" width="400" height="685"><figcaption>Figure adapted from <a href="https://openai.com/blog/chatgpt/?ref=assemblyai.com">source</a></figcaption></figure><h2 id="performance-evaluation"><a href="#performance-evaluation" class="anchor">#</a>Performance Evaluation</h2><p>Because the model is trained on human labelers input, <strong>the core part of the evaluation is also based on human input</strong>, i.e. it takes place by having labelers rate the quality of the model outputs. To avoid overfitting to the judgment of the labelers involved in the training phase, the test set uses prompts from held-out OpenAI customers which are not represented in the training data.</p><p>The model is evaluated on three high-level criteria:</p><ul><li>Helpfulness: judging the model’s ability to follow user instructions, as well as infer instructions.</li><li>Truthfulness: judging the model’s tendency for hallucinations (making up facts) on closed-domain tasks. The model is evaluated on the <a href="https://arxiv.org/abs/2109.07958?ref=assemblyai.com">TruthfulQA dataset</a>.</li><li>Harmlessness: the labelers evaluate whether the model’s output is appropriate, denigrates a protected class, or contains derogatory content. The model is also benchmarked on the <a href="https://allenai.org/data/real-toxicity-prompts?ref=assemblyai.com">RealToxicityPrompts</a> and <a href="https://aclanthology.org/2020.emnlp-main.154/?ref=assemblyai.com">CrowS-Pairs</a> datasets.</li></ul><p>The model is also evaluated for <a href="https://en.wikipedia.org/wiki/Zero-shot_learning?ref=assemblyai.com">zero-shot performance</a> on traditional NLP tasks like question answering, reading comprehension, and summarization, on some of which <strong>the developers observed performance regressions compared to GPT-3</strong>. This is an example of an “alignment tax” where the RLHF-based alignment procedure comes at the cost of lower performance on certain tasks. </p><p>The performance regressions on these datasets can be greatly reduced with a trick called <strong>pre-train mix</strong>: during training of the PPO model via <a href="https://en.wikipedia.org/wiki/Gradient_descent?ref=assemblyai.com">gradient descent</a>, the gradient updates are computed by mixing the gradients of the SFT model and the PPO model.</p><h2 id="shortcomings-of-the-methodology"><a href="#shortcomings-of-the-methodology" class="anchor">#</a>Shortcomings of the methodology</h2><p>A very clear limitation of the methodology, as discussed in the <a href="https://arxiv.org/pdf/2203.02155.pdf?ref=assemblyai.com">InstructGPT paper</a> (on which ChatGPT is based, according to its creators) is the fact that, in the process of aligning language models with human intentions, the data for fine-tuning the models is influenced by an intricate variety of subjective factors, including:</p><ul><li>The preferences of the labelers who produce the demonstration data.</li><li>The researchers who design the study and write the labeling instructions.</li><li>The choice of prompts crafted by the developers or provided by the OpenAI customers.</li><li>The labelers bias is both included in the reward model training (by ranking outputs) <strong>and in the model evaluation</strong>.</li></ul><p>In particular, the authors point out the obvious fact that the labelers and researchers taking part in the training process may not be representative of all potential end users of the language model.</p><p>Apart from this clear “intrinsic” limitation, we want to point out a few other possible shortcomings of the method, problems not explicitly addressed, as well as some open questions:</p><p><strong>Lack of control study</strong>: The <a href="https://arxiv.org/pdf/2203.02155.pdf?ref=assemblyai.com">reported results</a> measure the performance of the final PPO model, taking the SFT model as the baseline. This can be misleading: how can we know the improvements are actually due to the RLHF? A proper (yet, expensive) control study would consist in investing the exact same amount of labeler-hours as those used to train the reward model into creating a larger curated SFT dataset with high-quality demonstration data. One would then be in the position to objectively measure the performance improvements of the RLHF methodology versus the supervised approach. In very simple terms, the lack of such a control study is leaving a fundamental question completely open: is RLHF actually doing a good job in aligning language models?</p><p><strong>Lack of ground truth for the comparison data</strong>: labelers can often disagree on the ranking of the model’s outputs. Technically speaking, the risk is to add a <em>high potential variance</em> to the comparison data without any ground truth.</p><p><strong>Human preferences are just not homogeneous</strong>: The RLHF method treats human preferences as if they were homogeneous and static. Assuming that all people share the same values would be an obvious stretch, at least on a large amount of topics of human knowledge. <a href="https://arxiv.org/pdf/2211.15006.pdf?ref=assemblyai.com">Some recent research</a> is starting to tackle this open problem differently.</p><p><strong>Prompt-stability testing for the reward model (RM)</strong>: There seem to be no experiments investigating the sensitivity of the reward model in terms of changes in the input prompt. If two prompts are syntactically different but are semantically equivalent, can the RM show significant differences in the ranking of the model outputs? In simpler terms, how much does the <em>quality of the prompt</em> matter for the RM? </p><p><strong>Wireheading-type issues</strong>: In RL approaches, the model can sometimes learn to manipulate its own reward system to achieve a desired outcome, leading to an “over optimized policy”. This can push the model recreating some patterns that for some unknown reason make the reward model score high (see Table 29 in <a href="https://arxiv.org/pdf/2009.01325.pdf?ref=assemblyai.com">this paper from OpenAI</a> for an explicit example of this behavior in language modeling). ChatGPT puts a patch on this with the KL penalty term in the reward function. Note that one is trying to optimize the RM <em>input</em> (i.e. the PPO output) in order to improve its <em>output</em> (the reward score), <em>all the while constraining the input itself to be not too far away from some reference input</em> (the SFT output). More details on the limitations of this approach in <a href="https://arxiv.org/pdf/2205.11275.pdf?ref=assemblyai.com">this recent preprint</a>.</p><!--kg-card-begin: html--><a id="references"></a><!--kg-card-end: html--><h2 id="selected-references-for-further-reading"><a href="#selected-references-for-further-reading" class="anchor">#</a>Selected references for further reading</h2><ul><li>The most relevant paper about the RLHF methodology used for ChatGPT is <a href="https://arxiv.org/pdf/2203.02155.pdf?ref=assemblyai.com">Training language models to follow instructions with human feedback</a>, which in fact details a model called InstructGPT, referred to by OpenAI as a "sibling model" to ChatGPT.</li><li>Anthropic published <a href="https://arxiv.org/pdf/2204.05862.pdf?ref=assemblyai.com">a detailed study</a> on the effectiveness of RLHF methods for finetuning language models to act as helpful and harmless assistants.</li><li>The paper <a href="https://arxiv.org/pdf/2009.01325.pdf?ref=assemblyai.com">Learning to summarize from Human Feedback</a> describes RLHF in the context of text summarization.</li><li><a href="https://arxiv.org/pdf/1707.06347.pdf?ref=assemblyai.com">Proximal Policy Optimization</a>: the PPO algorithm paper.</li><li><a href="https://arxiv.org/abs/1706.03741?ref=assemblyai.com">Deep reinforcement learning from human preferences</a> –was one of the earliest (Deep Learning) papers using human feedback in RL, in the context of Atari games.</li><li>Alternatives to OpenAI's RLHF have been proposed by DeepMind in <a href="https://arxiv.org/pdf/2209.14375.pdf?ref=assemblyai.com">Sparrow</a> and <a href="https://arxiv.org/abs/2203.11147?ref=assemblyai.com">GopherCite</a> papers.</li><li>A deep dive into the Alignment problem for language models is given in a (long) <a href="https://arxiv.org/abs/2112.00861?ref=assemblyai.com">paper</a> by Anthropic. Here's an <a href="https://www.lesswrong.com/posts/oBpebs5j5ngs3EXr5/a-summary-of-anthropic-s-first-paper-3?ref=assemblyai.com">excellent summary by Sam Ringer</a>. Anthropic also has <a href="https://github.com/anthropics/hh-rlhf?ref=assemblyai.com">an open source repository</a> (with accompanying paper) for RLHF.</li></ul><!--kg-card-begin: html--><div data-template="banner" class="gh-canvas">
  
</div><!--kg-card-end: html-->
    </section>

   
  </article>
</a>  </div>
</div></div>